In [1]:
%matplotlib qt

import numpy as np
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
import cPickle as pickle

from spimagine import volshow
from spimagine import EllipsoidMesh, Mesh
from scipy.spatial import ConvexHull

# %matplotlib inline
# %pylab inline
# %pylab.rcParams['figure.figsize'] = (10, 10)

<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


/Users/jug/Library/Enthought/Canopy_64bit/User/lib/python2.7/site-packages/pyopencl/__init__.py:207: CompilerWarning: Non-empty compiler output encountered. Set the environment variable PYOPENCL_COMPILER_OUTPUT=1 to see more.
  "to see more.", CompilerWarning)


could not open /Users/jug/.spimagine
<pyopencl.Device 'HD Graphics 4000' on 'Apple' at 0x1024400>


# Sample unit sphere
(see http://blog.marmakoide.org/?p=1)

In [4]:
n = 1024
 
golden_angle = np.pi * (3 - np.sqrt(5))
theta = golden_angle * np.arange(n)
z = np.linspace(1 - 1.0 / n, 1.0 / n - 1, n)
radius = np.sqrt(1 - z * z)
 
points = np.zeros((n, 3))
points[:,0] = radius * np.cos(theta)
points[:,1] = radius * np.sin(theta)
points[:,2] = z

# Plot it and compute neighborhood

In [5]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
xs = points[:,0]
ys = points[:,1]
zs = points[:,2]
ax.scatter(xs, ys, zs, c='r', marker='o')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

In [6]:
num_neighbors = 6
neighbors=[]
for i,p1 in enumerate(points):
    dists = []
    for j,p2 in enumerate(points):
        dists.append( [i, j, np.dot(p1,p2)] )
    sorted_dists = sorted(dists, key=lambda dists: -dists[2])
    for idx in range(1,1+num_neighbors):
        neighbors.append(sorted_dists[idx])

In [7]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')

xs = points[:,0]
ys = points[:,1]
zs = points[:,2]
ax.scatter(xs, ys, zs, c='y', marker='x')

for cidx in range(0,n,128):
    cx = points[cidx,0]
    cy = points[cidx,1]
    cz = points[cidx,2]
    ax.scatter(cx, cy, cz, c='r', marker='o')

    nOfC = neighbors[cidx*num_neighbors:(cidx+1)*num_neighbors]
    p = points[[nOfC[i][1] for i in range(num_neighbors)]]
    xs = p[:,0]
    ys = p[:,1]
    zs = p[:,2]
    ax.scatter(xs, ys, zs, c='b', marker='^')

ax.set_xlabel('X')
ax.set_ylabel('Y')
ax.set_zlabel('Z')

plt.show()

In [8]:
neighbors_of = np.ones([n,num_neighbors]) * -1
for i,p1 in enumerate(points):
    dists = []
    for j,p2 in enumerate(points):
        dists.append( [i, j, np.dot(p1,p2)] )
    sorted_dists = sorted(dists, key=lambda dists: -dists[2])
    for idx in range(1,1+num_neighbors):
        neighbors.append(sorted_dists[idx])
    for idx in range(num_neighbors):
        neighbors_of[i,idx] = sorted_dists[idx+1][1]

print neighbors_of

[[  3.00000000e+00   2.00000000e+00   5.00000000e+00   1.00000000e+00
    8.00000000e+00   6.00000000e+00]
 [  6.00000000e+00   4.00000000e+00   0.00000000e+00   9.00000000e+00
    3.00000000e+00   2.00000000e+00]
 [  7.00000000e+00   0.00000000e+00   1.00000000e+01   5.00000000e+00
    1.50000000e+01   4.00000000e+00]
 ..., 
 [  1.01600000e+03   1.02300000e+03   1.01300000e+03   1.01800000e+03
    1.00800000e+03   1.01900000e+03]
 [  1.01700000e+03   1.01900000e+03   1.02300000e+03   1.01400000e+03
    1.02000000e+03   1.02100000e+03]
 [  1.02000000e+03   1.02100000e+03   1.01800000e+03   1.02200000e+03
    1.01500000e+03   1.01700000e+03]]


# Get triangulation of sampled unit sphere via _ConvexHull_ 

In [9]:
hull = ConvexHull(points)
normals = np.zeros_like(points)
count = np.zeros_like(points[:,np.newaxis])
for i,ind in enumerate(hull.simplices):
    normals[ind,:] += hull.equations[i,:3]
    count[ind] += 1
normals = normals/count

image = np.zeros((10,10,10))
w = volshow(image, raise_window=True)
m = Mesh(vertices=points, normals = normals, indices=hull.simplices.flatten())
w.glWidget.add_mesh( m )

In [10]:
triangles = hull.simplices

# Store data to file... 

In [11]:
dictDataStorage = {
    'n'            : n,
    'points'       : points,
    'neighbors'    : neighbors,
    'neighbors_of' : neighbors_of,
    'triangles'    : triangles
}
with open('sphere_sampling_1024.pkl','w') as f:
    pickle.dump(dictDataStorage,f)